In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import kagglehub
path = kagglehub.dataset_download("hijest/englishrussian-dictionary-for-machine-translate")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'englishrussian-dictionary-for-machine-translate' dataset.
Path to dataset files: /kaggle/input/englishrussian-dictionary-for-machine-translate


In [4]:
import string
import re
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import pandas as pd

# Путь к вашему файлу
text = "/kaggle/input/englishrussian-dictionary-for-machine-translate/rus.txt"

# Чтение файла и обработка строк
with open(text, encoding='utf-8') as file:
    lines = file.read().split("\n")

pairs = []

for line in lines:
    parts = line.split("\t")
    if len(parts) >= 2:
        english, russian = parts[0], parts[1]
        # Цель — английский русский
        english = "[start] " + english + " [end]"
        pairs.append((russian, english))

In [5]:
df = pd.DataFrame(pairs)
df.head(20)

,0,1
0,Марш!,[start] Go. [end]
1,Иди.,[start] Go. [end]
2,Идите.,[start] Go. [end]
3,Здравствуйте.,[start] Hi. [end]
4,Привет!,[start] Hi. [end]
5,Хай.,[start] Hi. [end]
6,Здрасте.,[start] Hi. [end]
7,Здоро́во!,[start] Hi. [end]
8,Беги!,[start] Run! [end]
9,Бегите!,[start] Run! [end]


In [7]:
import random
print(random.choice(pairs))
print(len(pairs))

('Все трое засмеялись.', '[start] The three of them began to laugh. [end]')
363386


In [8]:
random.shuffle(pairs)
num_val_samples = int(0.2 * len(pairs))
num_train_samples = len(pairs) - 2 * num_val_samples
train_pairs = pairs[:num_train_samples]
val_pairs = pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = pairs[num_train_samples + num_val_samples:]

In [9]:
print(len(train_pairs))
print(len(val_pairs))
print(len(test_pairs))

218032
72677
72677


In [10]:
strip_chars = string.punctuation + "." + "," + "?" + "!"

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

In [11]:
custom_standardization(pairs)

<tf.Tensor: shape=(363386, 2), dtype=string, numpy=
array([[b'\xd0\x90\xd1\x82\xd0\xbc\xd0\xbe\xd1\x81\xd1\x84\xd0\xb5\xd1\x80\xd0\xbd\xd1\x8b\xd0\xb9 \xd0\xba\xd0\xb8\xd1\x81\xd0\xbb\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb4 \xd1\x80\xd0\xb0\xd1\x81\xd1\x82\xd0\xb2\xd0\xbe\xd1\x80\xd1\x8f\xd0\xb5\xd1\x82\xd1\x81\xd1\x8f \xd0\xb2 \xd0\xb2\xd0\xbe\xd0\xb4\xd0\xb5',
        b'start oxygen from the air dissolves in water end'],
       [b'\xd0\x9c\xd0\xbd\xd0\xb5 \xd0\xbd\xd0\xb5 \xd1\x83\xd0\xb4\xd0\xb0\xd0\xbb\xd0\xbe\xd1\x81\xd1\x8c \xd0\xbd\xd0\xb0\xd0\xb9\xd1\x82\xd0\xb8 \xd0\xb5\xd0\xb3\xd0\xbe \xd0\xb4\xd0\xbe\xd0\xbc',
        b'start i couldnt find his house end'],
       [b'\xd0\xa2\xd0\xbe\xd0\xbc \xd0\xb2\xd1\x80\xd0\xbe\xd0\xb4\xd0\xb5 \xd0\xbd\xd0\xb5\xd0\xbf\xd0\xbb\xd0\xbe\xd1\x85\xd0\xbe\xd0\xb9 \xd1\x87\xd0\xb5\xd0\xbb\xd0\xbe\xd0\xb2\xd0\xb5\xd0\xba',
        b'start tom seems to be a kind person end'],
       ...,
       [b'\xd0\x9c\xd1\x8d\xd1\x80\xd0\xb8 \xd0\xbd\xd0\xb5 \xd0\x